<a href="https://colab.research.google.com/github/carminesant1/antispam-topic_modelling/blob/main/spam_mail%26topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analisi e Classificazione delle Email per la Rilevazione di SPAM**


---


L'obiettivo del progetto è quello di classificare e-mail spam con una rete MLP. Inoltre le mail contrassegnate come spam saranno sottoposte all'algoritmo LDA (Latent Dirichlet Allocation) al fine di individuarne i topic con relativa misura di similarità coseno al fine di verificarne la somiglianaza semantica. Infine, per le mail classificate come "non-spam", il programma restituirà informazioni sulle organizzazioni che compongono il corpo della mail.

In [1]:
!git clone https://github.com/carminesant1/antispam-topic_modelling.git

Cloning into 'antispam-topic_modelling'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 1.82 MiB | 4.95 MiB/s, done.


#Import delle librerie


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
import gensim.corpora as corpora
from pprint import pprint
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Import del dataset

In [ ]:
df = pd.read_csv("/content/antispam-topic_modelling/spam_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


# Pre-processing del dataset

Per le finalità del progetto bisogna procedere all'estrazione delle sole colonne "text", contenente il corpo di ciascuna mail, e "label_num", la variabile target che distingue le mail "non-spam" da quelle contrassegnate come "spam".
Inoltre sarà effettuata la divisione del dataset in "train" e "test" al fine di valutare l'accurattezza del modello su dati che non hanno contribuito al "training" dello stesso.
Infine si procederà alla preparazione del dataset, trasformando un dato testuale in un array numerico, ricorrendo alla tecnica del TfIdf.



In [ ]:
df = df[["text", "label_num"]]
print(df)

                                                   text  label_num
0     Subject: enron methanol ; meter # : 988291\nth...          0
1     Subject: hpl nom for january 9 , 2001\n( see a...          0
2     Subject: neon retreat\nho ho ho , we ' re arou...          0
3     Subject: photoshop , windows , office . cheap ...          1
4     Subject: re : indian springs\nthis deal is to ...          0
...                                                 ...        ...
5166  Subject: put the 10 on the ft\nthe transport v...          0
5167  Subject: 3 / 4 / 2000 and following noms\nhpl ...          0
5168  Subject: calpine daily gas nomination\n>\n>\nj...          0
5169  Subject: industrial worksheets for august 2000...          0
5170  Subject: important online banking alert\ndear ...          1

[5171 rows x 2 columns]


In [ ]:
#train e test
df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
#clean dataset
import string
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

english_stopwords = stopwords.words("english")
nlp = spacy.load("en_core_web_sm")
punctuation = set(string.punctuation)

def data_cleaner(dataset):
    dataset_to_return = []
    for sentence in dataset:
        sentence = str(sentence).lower()
        for c in string.punctuation:
            sentence = sentence.replace(c, " ")
        document = nlp(sentence)
        sentence = " ".join(token.lemma_ for token in document if token.lemma_ is not None)
        sentence = " ".join(word for word in sentence.split() if word not in english_stopwords)
        sentence = re.sub(r"\d+", "", sentence)
        dataset_to_return.append(sentence)
    return dataset_to_return

def bow_tfidf(dataset, tfidf_vectorizer):
  if tfidf_vectorizer == None:
    tfidf_vectorizer = TfidfVectorizer()
    X = tfidf_vectorizer.fit_transform(dataset)
  else:
    X = tfidf_vectorizer.transform(dataset)
  return X.toarray(), tfidf_vectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(df_train)

                                                   text  label_num
125   Subject: re : cornhusker\njohn ,\ni have a dra...          0
4271  Subject: alert : spam prevention\nr 3 mov 3\ns...          1
378   Subject: hpl nom for august 11 , 2000\n( see a...          0
4534  Subject: calpine daily gas nomination\n>\nrick...          0
5019  Subject: enron / hpl nom for november 18 , 200...          0
...                                                 ...        ...
1453  Subject: extend deal\ni have flow at meter 989...          0
3741  Subject: save up to 89 % on ink + no shipping ...          1
3840  Subject: meoh upset\nthe earliest time product...          0
3846  Subject: chemisorb\ncorrea , %\n75 % off for a...          1
4871  Subject: nomination - 2 - 23 - 2000\nplease in...          0

[4136 rows x 2 columns]


In [ ]:
df_train_cleaned, vectorized = bow_tfidf(data_cleaner(df_train["text"]), None)
df_test_cleaned, vectorized = bow_tfidf(data_cleaner(df_test["text"]), vectorized)

#Modello

Sbilanciamento delle classi trattato con parametro class_weight = "balanced"

In [ ]:
df["label_num"].value_counts()

,count
label_num,
0,3672
1,1499


In [ ]:
rf = RandomForestClassifier(class_weight = "balanced")

In [ ]:
rf.fit(df_train_cleaned, df_train['label_num'])

RandomForestClassifier(class_weight='balanced')

In [ ]:
y_pred = rf.predict(df_test_cleaned)
print("\nClassification Report sul test set:\n", classification_report(df_test['label_num'], y_pred))


Classification Report sul test set:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       737
           1       0.95      0.98      0.97       298

    accuracy                           0.98      1035
   macro avg       0.97      0.98      0.98      1035
weighted avg       0.98      0.98      0.98      1035



#Topic

Come da presentazione, un altro importante task del progetto è quello di andare ad individuare i topic delle mail contrassegnate come "spam". Per fare ciò ho preso in considerazione solo le osservazioni con valore 1 sulla variabile "label_num", sottoponendo i corpi delle mail ad una fase di preprocessing, questa volta utilizzando la tecnica "bag-of-word", compatibile con l'algoritmo LDA implementato nella libreria gensim.

In [ ]:
def sent_to_words(items):
  for item in items:
    yield(simple_preprocess(item, deacc=True))

def remove_stopwords(texts):
  return [[word for word in words if word not in stop_words and len(word) >= 5] for words in texts]

data_words = list(sent_to_words(df['text'][df['label_num']==1]))
data_words = remove_stopwords(data_words)

In [ ]:
print(data_words)

[['subject', 'photoshop', 'windows', 'office', 'cheap', 'trending', 'abasements', 'darer', 'prudently', 'fortuitous', 'undergone', 'lighthearted', 'charm', 'orinoco', 'taster', 'railroad', 'affluent', 'pornographic', 'cuvier', 'irvin', 'parkhouse', 'blameworthy', 'chlorophyll', 'robed', 'diagrammatic', 'fogarty', 'clears', 'bayda', 'inconveniencing', 'managing', 'represented', 'smartness', 'hashish', 'academies', 'shareholders', 'unload', 'badness', 'danielson', 'caffein', 'spaniard', 'chargeable', 'levin'], ['subject', 'looking', 'medication', 'source', 'difficult', 'material', 'condition', 'better', 'enough', 'excuse', 'found', 'simpliest', 'medication', 'perscription', 'delivery', 'private', 'secure', 'better', 'rightly', 'pound', 'squint', 'million', 'anything', 'erection', 'treatment', 'pills', 'depressant', 'pills', 'weight', 'splicings', 'bombahakcx', 'knowledge', 'human', 'power', 'synonymous', 'quality', 'stuff', 'rates', 'moneyback', 'guarantee', 'nature', 'sufficeth', 'autho

In [ ]:
id2word = corpora.Dictionary(data_words)#dizionario di parole
corpus = [id2word.doc2bow(text) for text in data_words]#bag of word

In [ ]:
num_topics = 3

lda_model = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word, #accetta solo bag of words?
                                       num_topics = num_topics,
                                       passes = 3
                                       )
pprint(lda_model.print_topics())

[(0,
  '0.008*"height" + 0.007*"subject" + 0.007*"pills" + 0.006*"width" + '
  '0.005*"align" + 0.004*"color" + 0.003*"border" + 0.003*"viagra" + '
  '0.003*"cialis" + 0.003*"center"'),
 (1,
  '0.011*"subject" + 0.009*"company" + 0.006*"statements" + '
  '0.005*"information" + 0.005*"price" + 0.004*"please" + 0.004*"within" + '
  '0.004*"report" + 0.004*"stock" + 0.003*"securities"'),
 (2,
  '0.014*"subject" + 0.004*"email" + 0.003*"money" + 0.003*"please" + '
  '0.003*"account" + 0.003*"information" + 0.002*"click" + 0.002*"online" + '
  '0.002*"company" + 0.002*"price"')]


Per valutare l'omogeneità semantica dei topic estratti ho deciso di utilizzare la similarità coseno, attraversano prima una fase di creazione di vettori per ciascun topic, considerando le prime 10 parole che li caratterizzano

In [ ]:
#lista di liste, ciascuna lista contiene le prime 10 parole caratterizzanti ciascun topic
list_topic = []
for topic in range(lda_model.num_topics):
  i_topic = lda_model.show_topic(topic, 10)
  word_i_topic = []
  for word in range(0,10):
    word_i_topic.append((i_topic[word][0]))
  list_topic.append(word_i_topic)

In [ ]:
#creare un vocabolario unificato con le prime 10 parole dei topic
vocabulary = list(set(word for topic in list_topic for word in topic))
#creare vettori per ciascun topic in base al vocabolario
topic_vectors = []
for topic in list_topic:
    #conto quante volte ogni parola del vocabolario appare nel topic
    vector = [topic.count(word) for word in vocabulary]
    topic_vectors.append(vector)
# calcolo la similarità coseno tra i vettori dei topic
topic_vectors = np.array(topic_vectors)  # Converti in un array numpy
cosine_similarities = cosine_similarity(topic_vectors)
# Matrice di similarità coseno
print("Matrice di similarità coseno tra i topic:")
print(cosine_similarities)


Matrice di similarità coseno tra i topic:
[[1.  0.1 0.1]
 [0.1 1.  0.5]
 [0.1 0.5 1. ]]


#Prova Programma

Inserire in input la mail.

Se il programma la classifica come "spam", ricerca anche il topic preponderante all'interno della stessa, restituendo la lista delle parole che più caratterizzano il topic al fine di darne un'interpretazione.
Se la mail è "non-spam" stampa le organizzazioni citate all'interno del corpo

In [ ]:
while True:
  mail = input("Inserisci la mail: ")
  cleaned_mail = data_cleaner([mail])
  X_vectorized, _ = bow_tfidf(cleaned_mail, vectorized)
  prd = clf.predict(X_vectorized)
  label_mapping = {0: "Non Spam", 1: "Spam"}
  print(label_mapping[prd[0]])
  if label_mapping[prd[0]] == "Spam":
    document = id2word.doc2bow(simple_preprocess(mail))#non è necessario eliminare le stop word perchè non ci sono nel train
    list_index = []
    list_score = []
    for index, score in lda_model[document]:
      list_index.append(index)
      list_score.append(score)
    list_score_sorted = sorted(list_score, reverse = True)
    max_score = list_score_sorted[0]
    max_score_index = list_score.index(max_score)
    print(f"Topic: {list_index[max_score_index]}, composto da {list_topic[max_score_index]}")
  else:
    doc_mail = nlp(mail)
    for token in doc_mail:
      if str(token.ent_type_) == "ORG":
          print(f"{str(token)}: {str(token.ent_type_)}")

